In [ ]:
from fhirclient.r4models import structuredefinition as SD
from pathlib import Path
from json import dumps, loads, load, JSONDecodeError  
import tarfile
from requests import get
from pprint import pprint

In [ ]:

package_path = Path.cwd() / 'tarfiles'/'package.tgz'  #get_si(path)

def get_si(package_path):
    sd_list =[]
    with tarfile.open(package_path, mode='r') as tf:
        #pprint(tf.getnames())
        for n in sorted(tf.getnames()):
            if n.startswith("package/StructureDefinition"):
                f = tf.extractfile(n)
                r = f.read()
                sd = loads(r)
                print(sd['id'])
                sd_list.append(sd)
    return sd_list

   
def get_si3(path):
    tf = get(f'{path}')
    return tf

"e.g. https://build.fhir.org/ig/HL7/davinci-deqm/package.tgz" 
ig_package_tar_path = 'http://build.fhir.org/ig/HL7/US-Core/package.tgz'

tf= get_si3(ig_package_tar_path) # get from remote server

package_path.write_bytes(tf.content)    #save in temp file
    
    
sd_list = get_si(package_path) #unpack from file

pprint([i["id"] for i in sd_list])

In [35]:
for sd_dict in sd_list:

        sd = SD.StructureDefinition(sd_dict)
       
        profile_id = sd.id
        print(f'### {profile_id}')

        for i,e in enumerate(sd.snapshot.element):
            #print(f'- {e.path}')
            if i==0: print(f'- {e.id}')
            if e.mustSupport:
                try:
                 print(f'   - {e.id} is min== {e.min} with must support == {e.mustSupport} element with type = {[j.code for j in e.type]}\n')
                except TypeError:
                    print ("   - no type (either a backbone or defined as pattern)\n")

### head-occipital-frontal-circumference-percentile
- Observation
   - Observation.status is min== 1 with must support == True element with type = ['code']

   - Observation.category is min== 1 with must support == True element with type = ['CodeableConcept']

   - Observation.category:VSCat is min== 1 with must support == True element with type = ['CodeableConcept']

   - Observation.category:VSCat.coding is min== 1 with must support == True element with type = ['Coding']

   - Observation.category:VSCat.coding.system is min== 1 with must support == True element with type = ['uri']

   - Observation.category:VSCat.coding.code is min== 1 with must support == True element with type = ['code']

   - Observation.code is min== 1 with must support == True element with type = ['CodeableConcept']

   - Observation.subject is min== 1 with must support == True element with type = ['Reference']

   - Observation.effective[x] is min== 1 with must support == True element with type = ['dateTime', '